In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

In [2]:
# Load the data
data = pd.read_csv("train.csv", low_memory=False)
data.drop_duplicates() # remove duplicates

,Characteristics.LotFeatures,Characteristics.LotSizeSquareFeet,ImageData.c1c6.summary.bathroom,ImageData.c1c6.summary.exterior,ImageData.c1c6.summary.interior,ImageData.c1c6.summary.kitchen,ImageData.c1c6.summary.property,ImageData.features_reso.results,ImageData.q1q6.summary.bathroom,ImageData.q1q6.summary.exterior,...,Structure.FireplacesTotal,Structure.GarageSpaces,Structure.Heating,Structure.LivingArea,Structure.NewConstructionYN,Structure.ParkingFeatures,Structure.Rooms.RoomsTotal,Structure.YearBuilt,Tax.Zoning,UnitTypes.UnitTypeType
0,NaN,3200.0,NaN,NaN,NaN,NaN,NaN,"['Cooling.CeilingFans', 'Flooring.Carpet', 'Fl...",NaN,NaN,...,NaN,NaN,NaN,NaN,False,NaN,NaN,0.0,commr,NaN
1,NaN,NaN,NaN,3.8,3.7,3.8,3.8,"['Appliances.GasRange', 'Appliances.Range', 'A...",NaN,4.2,...,NaN,NaN,['natural gas'],3175.0,False,['off alley'],6.0,NaN,NaN,NaN
2,NaN,18750.0,NaN,NaN,NaN,NaN,NaN,"['Flooring.Carpet', 'ParkingFeatures.Garage', ...",NaN,NaN,...,NaN,NaN,NaN,NaN,False,NaN,NaN,1926.0,commr,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['CommunityFeatures.Lake', 'WaterfrontFeatures...",NaN,NaN,...,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,agric,NaN
4,"['horses allowed', 'paddock', 'pond(s)']",NaN,NaN,3.6,NaN,NaN,3.6,"['Appliances.Refrigerator', 'Appliances.Microw...",NaN,4.2,...,NaN,NaN,['other'],0.0,False,"['unassigned', 'off street']",4.0,2006.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107432,NaN,NaN,NaN,3.8,NaN,NaN,3.8,"['Levels.One', 'ParkingFeatures.Attached', 'Pa...",NaN,4.1,...,NaN,2.0,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN
107433,NaN,NaN,2.8,NaN,3.6,3.1,3.1,"['Appliances.Dishwasher', 'Appliances.Stainles...",3.2,NaN,...,0.0,NaN,"['natural gas', 'forced air']",0.0,False,"['assigned', 'off street']",9.0,1963.0,NaN,NaN
107434,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['ExteriorFeatures.PrivateYard', 'ExteriorFeat...",NaN,NaN,...,NaN,2.5,NaN,NaN,False,['driveway'],NaN,NaN,NaN,NaN
107435,NaN,NaN,NaN,NaN,NaN,NaN,NaN,['ParkingFeatures.ParkingLot'],NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
# DATA CLEANING OF STORIES


# Map the 'ImageData.style.stories.summary.label' column to numeric values
story_mapping = {
    '1_story': 1,
    '1.5_stories': 1.5,
    '2_stories': 2,
    '2.5_stories': 2.5,
    '3_stories_or_more': 3
}


# print(data['ImageData.style.stories.summary.label'].unique())
data['ImageData.style.stories.summary.label'] = data['ImageData.style.stories.summary.label'].map(story_mapping)
print(data['ImageData.style.stories.summary.label'].unique())

[nan 2.  1.  1.5 2.5 3. ]


In [4]:
data['Structure.YearBuilt'].unique()

array([   0.,   nan, 1926., 2006., 1967., 1990., 1976., 1940., 2007.,
       1969., 1928., 1946., 1930., 1965., 1927., 1968., 1958., 2022.,
       1895., 1952., 1948., 1978., 1899., 1987., 2023., 1985., 2003.,
       1960., 1910., 1925., 1999., 1959., 2021., 1950., 1894., 1905.,
       1964., 1935., 1896., 1920., 1880., 2009., 1923., 1901., 1988.,
       1900., 1907., 1911., 1889., 2000., 1974., 1970., 1924., 1893.,
       1961., 1914., 1954., 2004., 1909., 1939., 1903., 1955., 1912.,
       2019., 1918., 1890., 1951., 2005., 1971., 1995., 1908., 1984.,
       1932., 2008., 2001., 1887., 1997., 1962., 1885., 1874., 1931.,
       2024., 2012., 1953., 1994., 1941., 1947., 1957., 1863., 1949.,
       1917., 1956., 1986., 1898., 1992., 1929., 1973., 1972., 1884.,
       1996., 1938., 1855., 1998., 1852., 1888., 1892., 1913., 1906.,
       1915., 1975., 1989., 1963., 1919., 1856., 1943., 2011., 2013.,
       2020., 1981., 1904., 2002., 1982., 1979., 1977., 1980., 1922.,
       1875., 2018.,

In [5]:
# # replace edge case 
# data['Structure.YearBuilt'] = data['Structure.YearBuilt'].replace(190., np.nan)

# # convert 0 to nan on Structure.YearBuilt to remove 0 values
# data['Structure.YearBuilt'] = data['Structure.YearBuilt'].replace(0, np.nan)


In [6]:
#data segmentation
columnes_numeriques = data.select_dtypes(include=['number'])  # Selecciona columnes numèriques
columnes_categoriques = data.select_dtypes(exclude=['number'])  # Selecciona columnes no numèriques (categòriques)

In [7]:
# Calcular el IQR para cada columna numérica
for columna in data.select_dtypes(include=['number']).columns:
    no_nans = columnes_numeriques.dropna(subset=columna)

    Q1 = no_nans[columna].quantile(0.05)
    Q3 = no_nans[columna].quantile(0.95)
    IQR = Q3 - Q1
    limite_inferior = Q1 - 1.5 * IQR
    limite_superior = Q3 + 1.5 * IQR
    
    # Filtrar las filas donde los valores están dentro de los límites
    columnes_numeriques = no_nans[(no_nans[columna] >= limite_inferior) & (no_nans[columna] <= limite_superior)]

columnes_numeriques

,Characteristics.LotSizeSquareFeet,ImageData.c1c6.summary.bathroom,ImageData.c1c6.summary.exterior,ImageData.c1c6.summary.interior,ImageData.c1c6.summary.kitchen,ImageData.c1c6.summary.property,ImageData.q1q6.summary.bathroom,ImageData.q1q6.summary.exterior,ImageData.q1q6.summary.interior,ImageData.q1q6.summary.kitchen,...,Structure.BathroomsFull,Structure.BathroomsHalf,Structure.BedroomsTotal,Structure.BelowGradeFinishedArea,Structure.BelowGradeUnfinishedArea,Structure.FireplacesTotal,Structure.GarageSpaces,Structure.LivingArea,Structure.Rooms.RoomsTotal,Structure.YearBuilt


In [8]:
# canviar els NaN per la mitjana de la columna
columnes_numeriques = columnes_numeriques.apply(lambda col: col.fillna(col.mean(skipna=True)) if col.dtype in ['float64', 'int64'] else col)
columnes_numeriques

,Characteristics.LotSizeSquareFeet,ImageData.c1c6.summary.bathroom,ImageData.c1c6.summary.exterior,ImageData.c1c6.summary.interior,ImageData.c1c6.summary.kitchen,ImageData.c1c6.summary.property,ImageData.q1q6.summary.bathroom,ImageData.q1q6.summary.exterior,ImageData.q1q6.summary.interior,ImageData.q1q6.summary.kitchen,...,Structure.BathroomsFull,Structure.BathroomsHalf,Structure.BedroomsTotal,Structure.BelowGradeFinishedArea,Structure.BelowGradeUnfinishedArea,Structure.FireplacesTotal,Structure.GarageSpaces,Structure.LivingArea,Structure.Rooms.RoomsTotal,Structure.YearBuilt


In [9]:
# normalize all numeric columns
scaler = MinMaxScaler()
columnes_numeriques = pd.DataFrame(scaler.fit_transform(columnes_numeriques), columns=columnes_numeriques.columns)
columnes_numeriques

ValueError: Found array with 0 sample(s) (shape=(0, 25)) while a minimum of 1 is required by MinMaxScaler.

In [ ]:
columnes_categoriques.head()

In [ ]:
columnes_numeriques.to_csv('columnes_numeriques.csv', index=False)
columnes_categoriques.to_csv('columnes_categoriques.csv', index=False)